<a href="https://colab.research.google.com/github/2403a52251-dot/NLP/blob/main/2403a52251_As6_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

task 1 : LDA with Sample Data and BoW

In [176]:
import pandas as pd
data=pd.read_excel('LDA-Data.xlsx')

display(data.head())

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [177]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)

    text = text.lower()

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokenized_words = word_tokenize(text)

    filtered_words = [word for word in tokenized_words if word not in stop_words]

    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [178]:
data['clean_News'] = data['News'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(data[['clean_News']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                       clean_News
0      virat scored century match
1                    bjp election
2       bumra took 5 wicket match
3  congress form state government


In [179]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(data['clean_News'])

In [180]:
import pandas as pd
feature_names = count_vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [181]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=2, random_state=42)

In [182]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")

        topic_weights = model.components_[topic_idx]

        sorted_indices = topic_weights.argsort()[::-1]

        top_indices = sorted_indices[:num_top_words]

        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [183]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
form government congress state election bjp match wicket bumra took 

Topic 1:
match virat century scored took bumra wicket bjp election state 


In [184]:
document_topics = LDA.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                       clean_News  topic
0      virat scored century match      1
1                    bjp election      0
2       bumra took 5 wicket match      1
3  congress form state government      0


task2 : LDA with Kaggle Data and BoW

In [195]:
import pandas as pd
df = pd.read_csv('/content/arxiv_data_210930-054931.csv', engine='python', nrows=1000)
df.head()

,terms,titles,abstracts
0,['cs.LG'],Multi-Level Attention Pooling for Graph Neural...,Graph neural networks (GNNs) have been widely ...
1,"['cs.LG', 'cs.AI']",Decision Forests vs. Deep Networks: Conceptual...,Deep networks and decision forests (such as ra...
2,"['cs.LG', 'cs.CR', 'stat.ML']",Power up! Robust Graph Convolutional Network v...,Graph convolutional networks (GCNs) are powerf...
3,"['cs.LG', 'cs.CR']",Releasing Graph Neural Networks with Different...,With the increasing popularity of Graph Neural...
4,['cs.LG'],Recurrence-Aware Long-Term Cognitive Network f...,Machine learning solutions for pattern classif...


In [196]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)

    text = text.lower()

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokenized_words = word_tokenize(text)

    filtered_words = [word for word in tokenized_words if word not in stop_words]

    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [197]:
df['clean_summaries_pipeline'] = df['abstracts'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(df[['clean_summaries_pipeline']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                            clean_summaries_pipeline
0  graph neural network gnns widely used learn ve...
1  deep network decision forest random forest gra...
2  graph convolutional network gcns powerful tool...
3  increasing popularity graph neural network gnn...
4  machine learning solution pattern classificati...


In [198]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(df['clean_summaries_pipeline'])

In [199]:
import pandas as pd
feature_names = count_vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   01  08  10  100  10000  105  11  12  120  13  ...  xray  year  yield  \
0   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
1   0   0   0    1      1    0   0   0    0   0  ...     0     0      0   
2   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
3   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
4   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
5   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
6   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
7   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
8   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   
9   0   0   0    0      0    0   0   0    0   0  ...     0     0      0   

   yielded  yielding  youtube  youtubevos  zero  zeroone  zeroshot  
0        0         0        0           0     0        0         0  
1        0         0        0       

In [200]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)


LatentDirichletAllocation(n_components=2, random_state=42)

In [201]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")

        topic_weights = model.components_[topic_idx]

        sorted_indices = topic_weights.argsort()[::-1]

        top_indices = sorted_indices[:num_top_words]

        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [202]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
model language task natural image learning method data transformer processing 

Topic 1:
graph data network learning model method neural node representation task 


In [203]:
document_topics = LDA.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                            clean_summaries_pipeline  topic
0  graph neural network gnns widely used learn ve...      1
1  deep network decision forest random forest gra...      1
2  graph convolutional network gcns powerful tool...      1
3  increasing popularity graph neural network gnn...      1
4  machine learning solution pattern classificati...      1


task 3: NMF with Sample Data and BoW

In [185]:
from sklearn.decomposition import NMF

num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)

nmf_model.fit(doc_term_matrix)
print("NMF model initialized and fitted successfully.")



NMF model initialized and fitted successfully.


In [186]:
print(f"\nTop {num_top_words} words per topic (NMF):")
display_topics(nmf_model, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic (NMF):

Topic 0:
match virat took scored wicket bumra century bjp election state 

Topic 1:
state form congress government election bjp took virat wicket scored 


In [187]:
document_topics = nmf_model.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                       clean_News  topic
0      virat scored century match      0
1                    bjp election      1
2       bumra took 5 wicket match      0
3  congress form state government      1


task 4: NMF with Kaggle Data and BoW

In [204]:

from sklearn.decomposition import NMF

num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)

nmf_model.fit(doc_term_matrix)
print("NMF model initialized and fitted successfully.")


NMF model initialized and fitted successfully.


In [205]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic (NMF):")
display_topics(nmf_model, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic (NMF):

Topic 0:
model data learning method task language image network natural performance 

Topic 1:
graph network node data learning neural representation method classification structure 


In [206]:
document_topics = nmf_model.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                            clean_summaries_pipeline  topic
0  graph neural network gnns widely used learn ve...      1
1  deep network decision forest random forest gra...      0
2  graph convolutional network gcns powerful tool...      1
3  increasing popularity graph neural network gnn...      0
4  machine learning solution pattern classificati...      0


task 5:NMF with Sample Data and TFIDF

In [188]:
import pandas as pd
data = pd.read_excel('/content/LDA-Data.xlsx')
display(data.head())

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [189]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
def nltk_preprocessing_pipeline(text):
  lemmatizer = WordNetLemmatizer()
  stop_words = set(stopwords.words('english'))
  text = re.sub(r'http\S+|www\S+|https\S+','',text, flags = re.MULTILINE)
  text = re.sub(r'<.*?>', '', text)
  text = re.sub(r'@\w+', '', text)
  text = re.sub(r'#\w+', '', text)
  text = text.lower()
  emoji_pattern = re.compile(
          "["
          "\U0001F600-\U0001F64F"
          "\U0001F300-\U0001F5FF"
          "\U0001F680-\U0001F6FF"
          "\U0001F1E0-\U0001F1FF"
          "\U00002702-\U000027B0"
          "\U000024C2-\U0001F251"
          "]+", flags=re.UNICODE
      )
  text = emoji_pattern.sub(r'', text)
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  text = re.sub(r'\s+', ' ', text).strip()
  tokenized_words = word_tokenize(text)
  filtered_words = [word for word in tokenized_words if word not in stop_words]
  lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
  clean_summary = ' '.join(lemmatized_words)
  return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [190]:
data['clean_News'] = data['News'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(data[['clean_News']].head())




Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                       clean_News
0      virat scored century match
1                    bjp election
2       bumra took 5 wicket match
3  congress form state government


In [191]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
           lowercase=True,
           token_pattern=r'\b[a-zA-Z]{2,}\b',
           max_features=3000)
doc_term_matrix = vectorizer.fit_transform(data['clean_News'])

In [192]:
from sklearn.decomposition import NMF
num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(doc_term_matrix)
print("NMF model initialized and fitted successfully.")

NMF model initialized and fitted successfully.


In [193]:
print(f"\nTop {num_top_words} words per topic (NMF):")
display_topics(nmf_model, count_vectorizer.get_feature_names_out(), num_top_words)



Top 10 words per topic (NMF):

Topic 0:
match bumra wicket took virat scored century state government election 

Topic 1:
election bjp form government state congress took virat wicket scored 


In [194]:
document_topics = nmf_model.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                       clean_News  topic
0      virat scored century match      0
1                    bjp election      1
2       bumra took 5 wicket match      0
3  congress form state government      1


TASK 6 :NMF FOR KAGGLE-DATA USING TF-IDF

In [207]:
import pandas as pd
df = pd.read_csv('/content/arxiv_data_210930-054931.csv', engine='python', on_bad_lines='warn')
display(data.head())

,News,clean_News,topic
0,Virat scored century in match,virat scored century match,0
1,BJP won in elections,bjp election,1
2,Bumra took 5 wicket in a match,bumra took 5 wicket match,0
3,Congress form state government,congress form state government,1


In [208]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = text.lower()
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokenized_words = word_tokenize(text)
    filtered_words = [word for word in tokenized_words if word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [210]:
df['clean_summaries_pipeline'] = df['abstracts'].head(100).apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(df[['clean_summaries_pipeline']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                            clean_summaries_pipeline
0  graph neural network gnns widely used learn ve...
1  deep network decision forest random forest gra...
2  graph convolutional network gcns powerful tool...
3  increasing popularity graph neural network gnn...
4  machine learning solution pattern classificati...


In [211]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
           lowercase=True,
           token_pattern=r'\b[a-zA-Z]{2,}\b',
           max_features=3000)
df['clean_summaries_pipeline'] = df['clean_summaries_pipeline'].fillna('')
doc_term_matrix = vectorizer.fit_transform(df['clean_summaries_pipeline'])

In [212]:
from sklearn.decomposition import NMF
num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(doc_term_matrix)
print("NMF model initialized and fitted successfully.")

NMF model initialized and fitted successfully.


In [213]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic (NMF):")
display_topics(nmf_model, vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic (NMF):

Topic 0:
graph node gnns network learning representation neural graphstructured data method 

Topic 1:
data model network learning structured feature deep method space task 


In [214]:
document_topics = nmf_model.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                            clean_summaries_pipeline  topic
0  graph neural network gnns widely used learn ve...      0
1  deep network decision forest random forest gra...      1
2  graph convolutional network gcns powerful tool...      0
3  increasing popularity graph neural network gnn...      1
4  machine learning solution pattern classificati...      1
